## Decision Tree

#### Steps
- Pre-process data
    - Dropping not useful features
    - Filling the missing values(Data imputation)
- Creating Binary Decision Tree

In [1]:
import pandas as pd
import numpy as np

In [109]:
df = pd.read_csv('./Desktop/ML Data/Train.csv')
data = pd.read_csv('./Desktop/ML Data/Test.csv')

In [110]:
df.head(n=5)
data.head(n=5)

,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1.0,"Flynn, Mr. John Irwin (""Irving"")",male,36.0,0.0,0.0,PC 17474,26.3875,E25,S,5,NaN,"Brooklyn, NY"
1,3.0,"Sage, Miss. Constance Gladys",female,NaN,8.0,2.0,CA. 2343,69.5500,NaN,S,NaN,NaN,NaN
2,1.0,"Rood, Mr. Hugh Roscoe",male,NaN,0.0,0.0,113767,50.0000,A32,S,NaN,NaN,"Seattle, WA"
3,2.0,"Gillespie, Mr. William Henry",male,34.0,0.0,0.0,12233,13.0000,NaN,S,NaN,NaN,"Vancouver, BC"
4,2.0,"Collander, Mr. Erik Gustaf",male,28.0,0.0,0.0,248740,13.0000,NaN,S,NaN,NaN,"Helsinki, Finland Ashtabula, Ohio"


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1009 entries, 0 to 1008
Data columns (total 14 columns):
pclass       1009 non-null float64
survived     1009 non-null float64
name         1009 non-null object
sex          1009 non-null object
age          812 non-null float64
sibsp        1009 non-null float64
parch        1009 non-null float64
ticket       1009 non-null object
fare         1008 non-null float64
cabin        229 non-null object
embarked     1008 non-null object
boat         374 non-null object
body         98 non-null float64
home.dest    582 non-null object
dtypes: float64(7), object(7)
memory usage: 110.4+ KB


In [83]:
columns_to_drop = ["name","parch","boat","body","home.dest","cabin","embarked","ticket"]

In [111]:
data_clean = df.drop(columns_to_drop, axis=1)
test_clean = data.drop(columns_to_drop, axis=1)

In [112]:
data_clean.head()
test_clean.head()

,pclass,sex,age,sibsp,fare
0,1.0,male,36.0,0.0,26.3875
1,3.0,female,NaN,8.0,69.5500
2,1.0,male,NaN,0.0,50.0000
3,2.0,male,34.0,0.0,13.0000
4,2.0,male,28.0,0.0,13.0000


In [113]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data_clean["sex"] = le.fit_transform(data_clean["sex"])
test_clean["sex"] = le.fit_transform(test_clean["sex"])

In [114]:
data_clean.head()
test_clean.head()

,pclass,sex,age,sibsp,fare
0,1.0,1,36.0,0.0,26.3875
1,3.0,0,NaN,8.0,69.5500
2,1.0,1,NaN,0.0,50.0000
3,2.0,1,34.0,0.0,13.0000
4,2.0,1,28.0,0.0,13.0000


In [116]:
data_clean.info()
test_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1009 entries, 0 to 1008
Data columns (total 6 columns):
pclass      1009 non-null float64
survived    1009 non-null float64
sex         1009 non-null int32
age         812 non-null float64
sibsp       1009 non-null float64
fare        1008 non-null float64
dtypes: float64(5), int32(1)
memory usage: 43.4 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 5 columns):
pclass    300 non-null float64
sex       300 non-null int32
age       234 non-null float64
sibsp     300 non-null float64
fare      300 non-null float64
dtypes: float64(4), int32(1)
memory usage: 10.6 KB


In [117]:
data_clean = data_clean.fillna(data_clean["age"].mean())
test_clean = test_clean.fillna(test_clean["age"].mean())

In [118]:
data_clean.info()
test_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1009 entries, 0 to 1008
Data columns (total 6 columns):
pclass      1009 non-null float64
survived    1009 non-null float64
sex         1009 non-null int32
age         1009 non-null float64
sibsp       1009 non-null float64
fare        1009 non-null float64
dtypes: float64(5), int32(1)
memory usage: 43.4 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 5 columns):
pclass    300 non-null float64
sex       300 non-null int32
age       300 non-null float64
sibsp     300 non-null float64
fare      300 non-null float64
dtypes: float64(4), int32(1)
memory usage: 10.6 KB


In [92]:
data_clean.loc[1]

pclass       2.0
survived     0.0
sex          1.0
age         39.0
sibsp        0.0
fare        26.0
Name: 1, dtype: float64

In [95]:
input_cols = ["pclass","sex","age","sibsp","fare"]
output_cols = ["survived"]

X = data_clean[input_cols]
Y = data_clean[output_cols]

print(X.shape, Y.shape)
print(type(X))

(1009, 5) (1009, 1)
<class 'pandas.core.frame.DataFrame'>


In [96]:
def entropy(col):
    
    counts = np.unique(col, return_counts=True)
    N = float(col.shape[0])
    
    ent = 0.0
    for ix in counts[1]:
        p = ix/N
        ent += (-1.0*p*np.log2(p))
    return ent

In [97]:
def divide_data(x_data,fkey,fval):
    #Work with Pandas Data Frames
    x_right = pd.DataFrame([],columns=x_data.columns)
    x_left = pd.DataFrame([],columns=x_data.columns)
    
    for ix in range(x_data.shape[0]):
        val = x_data[fkey].loc[ix]
        
        if val > fval:
            x_right = x_right.append(x_data.loc[ix])
        else:
            x_left = x_left.append(x_data.loc[ix])
            
    return x_left,x_right

In [98]:
def information_gain(x_data,fkey,fval):    
    left,right = divide_data(x_data,fkey,fval)
    
    #% of total samples are on left and right
    l = float(left.shape[0])/x_data.shape[0]
    r = float(right.shape[0])/x_data.shape[0]
    
    #All examples come to one side!
    if left.shape[0] == 0 or right.shape[0] ==0:
        return -1000000 #Min Information Gain
    
    i_gain = entropy(x_data.survived) - (l*entropy(left.survived)+r*entropy(right.survived))
    return i_gain

In [99]:
split = int(0.7*data_clean.shape[0])
train_data = data_clean[:split]
test_data = data_clean[split:]
test_data = test_data.reset_index(drop=True)

In [100]:
print(train_data.shape,test_data.shape)
print(type(train_data))

(706, 6) (303, 6)
<class 'pandas.core.frame.DataFrame'>


In [101]:
from sklearn.tree import DecisionTreeClassifier

In [102]:
sk_tree = DecisionTreeClassifier(criterion='gini',max_depth=5)

In [104]:
sk_tree.fit(data_clean[input_cols], data_clean[output_cols])

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [139]:
ans = sk_tree.predict(test_clean[input_cols])
ans.shape
final = pd.DataFrame(ans, columns=['survived'])
final.head()
final.index.name = 'Id'
print(final.index.name)
final.head()
final.to_csv('final.csv')

Id
